In [ ]:
import cv2
#영상 길이 몇초
# 동영상 불러오기
video_path = r"C:\Users\user\Downloads\Flat_0.01X_10sec.mp4"

cap = cv2.VideoCapture(video_path)

# 총 프레임 수
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

# FPS (Frame Per Second)
fps = cap.get(cv2.CAP_PROP_FPS)

# 동영상 전체 길이 (초 단위, 소수점 포함)
duration = frame_count / fps

print("총 프레임 수:", frame_count)
print("FPS:", fps)
print(f"동영상 길이: {duration:.3f} 초")

C:\Users\GCU\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe
